<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 09**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**

*   Continuar o estudo do analisador semântico
*   Entender o processo de verificação de tipos
*   Aplicar o design pattern Visitor para realizar o processo de verificação de tipos


In [ ]:
!pip install rply

**ATIVIDADE DA AULA PASSADA**

Na aula passada, aumentamos a gramática vista em aula para incluir declaração de variáveis com os tipos int e string, assim como um comando de atribuição. Foi solicitada a implementação de dois visitors:

(1) um visitor para montar a tabela de símbolos

(2) um visitor para decorar a árvore sintática. Suponha que NUMBER tenha o tipo int.


\<prog\> ::= \<var-decls\> \<atrib\>

\<var-decls\> ::= \<var-decl\> 

       | \<var-decl\> \<var-decls\>

\<var-decl\> ::= \<type\> ID ;

\<type\> ::= int | string 

\<atrib\> ::= ID = \<expression\>

\<expression\> ::= ID

      | NUMBER

      | \<expression\> "+" \<expression\>
 
      | \<expression\> "-" \<expression\>
 
      | \<expression\> "*" \<expression\>
 
      | \<expression\> "/" \<expression\>
 
      | "(" <expression> ")"

Inicialmente, vamos implementar um analisador léxico para esta gramática aumentada:

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

A seguir, temos a implementação das classes dos nós da árvore sintática, já com o método accept para receber os visitors:

In [ ]:
from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,atrib):
        self.decls = decls
        self.atrib = atrib

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp
        

    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass
    

class Sub(BinaryOp):
  pass
   

class Mul(BinaryOp):
  pass
    

class Div(BinaryOp):
  pass
    

A seguir, temos o analisador sintático:

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

@pg.production('prog : vardecls atrib')
def prog(p):
    return Prog(p[0],p[1])

@pg.production('vardecls : vardecl')
def expression_vardeclsOne(p):
    return VarDecls(p[0],None) 

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def expression_vardeclstring(p):
    return VarDecl(p[1].getstr(), p[0].getstr())

@pg.production('vardecl : INT ID SEMICOL')
def expression_vardeclint(p):
    return VarDecl(p[1].getstr(), p[0].getstr())

@pg.production('atrib : ID EQUALS expression')
def atrib(p):
    return Atrib(p[0].getstr(),p[2])


@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Aqui, temos a árvore sintática montada...

In [ ]:
arvore=parser.parse(lexer.lex('int x;string y;int z;z=x+y'))

Aqui, temos a implementação do visitor de montagem da tabela de símbolos:

In [ ]:
ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)
        
    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp
        

Finalmente, temos o teste do vísitor de montagem da tabela de símbolos:

In [ ]:
arvore.accept(SymbolTable())
for k, v in ST.items():
  print(k, v)

x int
y string
z int


**EXERCÍCIO**

Implementar um visitor para decorar a árvore sintática:

In [ ]:
class Decorator(Visitor):

    def visit_prog(self, i):
        i.atrib.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.id_decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)
        i.expr_decor_type=i.expr.decor_type

    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int"
        

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
          

    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

In [ ]:
arvore.accept(Decorator())

AttributeError: ignored

**EXERCÍCIO**

Implementar um visitor para verificar os tipos decorados:

In [ ]:
class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.atrib.accept(self)

    def visit_atrib(self, i):
        if i.id_decor_type!=i.expr_decor_type:
          raise AssertionError('type error')

    def visit_add(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')
          

    def visit_sub(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

    def visit_mul(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

    def visit_div(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

In [ ]:
arvore.accept(TypeVerifier())

AssertionError: ignored

**ATIVIDADE EAD**

Ler e fazer um pequeno resumo da máquina de P-Código:

https://homepages.cwi.nl/~steven/pascal/book/10pcode.html

